### Synthetic Data Backtest

Most of the functions below can be found under:

* Tool/backtest 
* Sample_data/make_data

A quick slideshow: [Seven Sins of Backtesting](https://newyork.qwafafew.org/wp-content/uploads/sites/4/2015/10/Luo_20150128.pdf)

The below is a code snippet for trading rules optimization.

The origin exercise required to use real-HFT data to deduce a certain outcome.. which I don't have.

Hence, I will just replace this exercise with multiprocessing methods (Parallelization!).

If you are keen on generating synthetic data for your research, copy the code snippets [Generate synthetic raw data](https://gist.github.com/boyboi86/5e00faf48f60abfdbe838fbdee269471) in my gist.

Contact: boyboi86@gmail.com

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import research as rs
from random import gauss
from itertools import product

%matplotlib inline

Num of CPU core:  4
Machine info:  Windows-10-10.0.18362-SP0
Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Numpy 1.18.5
Pandas 1.0.4


C:\Users\Wei_X\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<Figure size 1500x800 with 1 Axes>

In [2]:
dict0 = {'a':['1', '2'], 'b':['+', '*'], 'c':['!','A']}
for a in dict0['a']:
    for b in dict0['b']:
        for c in dict0['c']:
            print({'a':a, 'b':b, 'c':c})

{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': 'A'}
{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': 'A'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': 'A'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': 'A'}


**Note**

The above is scalar operation. Notice loop after loop after loop.

The below is vectorization operation, which is what we will do for Monte-Carlos.

The result is actually the same.

In [3]:
jobs = (dict(zip(dict0, i )) for i in product(*dict0.values()))
for i in jobs: print(i)

{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': 'A'}
{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': 'A'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': 'A'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': 'A'}


**Note**

Given the parameters below. It should look like the below.

In [4]:
forecast = [10,5,0,-5,-10]
half_life = [5,10,25,50,100]
loss_range = profit_range = np.linspace(.5,10,20)
sigma = [1]

coeff = dict({'forecast': forecast,
              'half_life': half_life,
              'sigma':sigma,
              'profit_range': profit_range,
              'loss_range':loss_range})
jobs = list(dict(zip(coeff, idx)) for idx in product(*coeff.values()))

print(len(jobs)) # 5 * 5 * 20 * 20 = forecast * half_life * loss * profit
out = pd.DataFrame(jobs)
out

10000


,forecast,half_life,sigma,profit_range,loss_range
0,10,5,1,0.5,0.5
1,10,5,1,0.5,1.0
2,10,5,1,0.5,1.5
3,10,5,1,0.5,2.0
4,10,5,1,0.5,2.5
...,...,...,...,...,...
9995,-10,100,1,10.0,8.0
9996,-10,100,1,10.0,8.5
9997,-10,100,1,10.0,9.0
9998,-10,100,1,10.0,9.5


**Note**

The below is the end product for a non-parallized Monte-Carlos Simulation, for optimizied Trading Rule Algorithm.

Primarily we will use it to create scenarios for our models to be backtest with.

There is another algorithm which MLfinlab has implemented, you may wish to read that as well.

[Backtesting by Campbell and Yan [2015]](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2345489)

In [5]:
def _opt_tr(coeff: dict = None,
            n_run: float = 100,
            max_period: int = 100,
            seed: int = 0):
    
    phi, output1 = 2 ** (-1/ coeff['half_life']), []
    for iter_ in range(int(n_run)):
        p, hold_period, count = seed, 0, 0
        while True:
            p = (1 - phi) * coeff['forecast'] + phi * p + coeff['sigma'] * gauss(0,1)
            cp = p - seed
            hold_period += 1
            if coeff['profit_range'] < cp or -coeff['loss_range'] > cp or hold_period > max_period:
                output1.append(cp)
                break
    mean, std = np.mean(output1), np.std(output1)
    return {'profit': coeff['profit_range'], 'loss': coeff['loss_range'], 'mean': mean, 'std': std, 'sharpe': mean/std}

def opt_tr(profit_range: list = np.linspace(.5, 10, 20),
          loss_range: list = np.linspace(.5, 10, 20),
          sigma: list = [1],
          n_run: float = 100,
          max_period: int = 100,
          forecast: list = [10,5,0,-5,-10], 
          half_life: list = [5,10,25,50,100],
          num_threads: int = 1):
    
    if profit_range is None:
        profit_range = np.linspace(0,10,21)
        
    if loss_range is None:
        loss_range = np.linspace(0,10,21)
        
    _coeff_list = []
    _coeff = dict({'forecast': forecast,
                   'half_life': half_life,
                   'sigma':sigma,
                   'profit_range': profit_range,
                   'loss_range':loss_range})
    
    _coeff = list(dict(zip(_coeff, idx )) for idx in product(*_coeff.values()))
    out = []
    for coeff in _coeff:
        jobs = [{'func': _opt_tr,
                   'coeff': coeff,
                   'n_run': n_run, 
                   'max_period': max_period}]
        
        if num_threads == 1:
            out_ = rs.process_jobs_(jobs)
            out.append(out_)
        else:
            out_ = rs.process_jobs(jobs, num_threads = num_threads)
            out.append(out_)

    return out

**Note**

Before parallization. Total runs for estimated 1,000,000 loops took only less than 1 min

* 2020-06-14 17:02:17.058014 (Beginning!)

* 2020-06-14 17:03:06.231600 (End!)

**Note**

Try to change the above code snippet to run parallelization.

Please do not use jupyter notbook, it won't reflect anything.

If you are like me, curious about strategy's mean-reversion velocity, you might want to read this research report.

[Critical-Line Algorithm for Portfolio Optimization by Bailey and Lopez [2013]](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2197616)

**Hint**

Vectorize iterable then try using "iterrows()" or "itertuples()"

In [6]:
df = rs.opt_tr(num_threads=1) #just 1 thread non-parallel

2020-06-15 12:31:07.099851 done after 0.00 mins
2020-06-15 12:31:07.099851 done after 0.00 mins
2020-06-15 12:31:07.099851 done after 0.00 mins
2020-06-15 12:31:07.099851 done after 0.00 mins
2020-06-15 12:31:07.099851 done after 0.00 mins
2020-06-15 12:31:07.099851 done after 0.00 mins
2020-06-15 12:31:07.099851 done after 0.00 mins
2020-06-15 12:31:07.099851 done after 0.00 mins
2020-06-15 12:31:07.114043 done after 0.00 mins
2020-06-15 12:31:07.116042 done after 0.00 mins
2020-06-15 12:31:07.117039 done after 0.00 mins
2020-06-15 12:31:07.118032 done after 0.00 mins
2020-06-15 12:31:07.118032 done after 0.00 mins
2020-06-15 12:31:07.119030 done after 0.00 mins
2020-06-15 12:31:07.119030 done after 0.00 mins
2020-06-15 12:31:07.119030 done after 0.00 mins
2020-06-15 12:31:07.120026 done after 0.00 mins
2020-06-15 12:31:07.121025 done after 0.00 mins
2020-06-15 12:31:07.121025 done after 0.00 mins
2020-06-15 12:31:07.122020 done after 0.00 mins
2020-06-15 12:31:07.122020 done after 0.

2020-06-15 12:31:07.288059 done after 0.00 mins
2020-06-15 12:31:07.291785 done after 0.00 mins
2020-06-15 12:31:07.294774 done after 0.00 mins
2020-06-15 12:31:07.296768 done after 0.00 mins
2020-06-15 12:31:07.299706 done after 0.00 mins
2020-06-15 12:31:07.300703 done after 0.00 mins
2020-06-15 12:31:07.301701 done after 0.00 mins
2020-06-15 12:31:07.302698 done after 0.00 mins
2020-06-15 12:31:07.304693 done after 0.00 mins
2020-06-15 12:31:07.307686 done after 0.00 mins
2020-06-15 12:31:07.312694 done after 0.00 mins
2020-06-15 12:31:07.314668 done after 0.00 mins
2020-06-15 12:31:07.318655 done after 0.00 mins
2020-06-15 12:31:07.321647 done after 0.00 mins
2020-06-15 12:31:07.323642 done after 0.00 mins
2020-06-15 12:31:07.324640 done after 0.00 mins
2020-06-15 12:31:07.326634 done after 0.00 mins
2020-06-15 12:31:07.330625 done after 0.00 mins
2020-06-15 12:31:07.333615 done after 0.00 mins
2020-06-15 12:31:07.335613 done after 0.00 mins
2020-06-15 12:31:07.337617 done after 0.

2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.673286 done after 0.00 mins
2020-06-15 12:31:07.683127 done after 0.00 mins
2020-06-15 12:31:07.684126 done after 0.00 mins
2020-06-15 12:31:07.686116 done after 0.

2020-06-15 12:31:08.056748 done after 0.00 mins
2020-06-15 12:31:08.056748 done after 0.00 mins
2020-06-15 12:31:08.056748 done after 0.00 mins
2020-06-15 12:31:08.056748 done after 0.00 mins
2020-06-15 12:31:08.056748 done after 0.00 mins
2020-06-15 12:31:08.056748 done after 0.00 mins
2020-06-15 12:31:08.072411 done after 0.00 mins
2020-06-15 12:31:08.072411 done after 0.00 mins
2020-06-15 12:31:08.072411 done after 0.00 mins
2020-06-15 12:31:08.072411 done after 0.00 mins
2020-06-15 12:31:08.072411 done after 0.00 mins
2020-06-15 12:31:08.072411 done after 0.00 mins
2020-06-15 12:31:08.072411 done after 0.00 mins
2020-06-15 12:31:08.090525 done after 0.00 mins
2020-06-15 12:31:08.093585 done after 0.00 mins
2020-06-15 12:31:08.093585 done after 0.00 mins
2020-06-15 12:31:08.093585 done after 0.00 mins
2020-06-15 12:31:08.093585 done after 0.00 mins
2020-06-15 12:31:08.093585 done after 0.00 mins
2020-06-15 12:31:08.093585 done after 0.00 mins
2020-06-15 12:31:08.093585 done after 0.

2020-06-15 12:31:08.723787 done after 0.00 mins
2020-06-15 12:31:08.727995 done after 0.00 mins
2020-06-15 12:31:08.731982 done after 0.00 mins
2020-06-15 12:31:08.735973 done after 0.00 mins
2020-06-15 12:31:08.740966 done after 0.00 mins
2020-06-15 12:31:08.743950 done after 0.00 mins
2020-06-15 12:31:08.747942 done after 0.00 mins
2020-06-15 12:31:08.750931 done after 0.00 mins
2020-06-15 12:31:08.752926 done after 0.00 mins
2020-06-15 12:31:08.753924 done after 0.00 mins
2020-06-15 12:31:08.755918 done after 0.00 mins
2020-06-15 12:31:08.757913 done after 0.00 mins
2020-06-15 12:31:08.759907 done after 0.00 mins
2020-06-15 12:31:08.761902 done after 0.00 mins
2020-06-15 12:31:08.765891 done after 0.00 mins
2020-06-15 12:31:08.767886 done after 0.00 mins
2020-06-15 12:31:08.769880 done after 0.00 mins
2020-06-15 12:31:08.772872 done after 0.00 mins
2020-06-15 12:31:08.774867 done after 0.00 mins
2020-06-15 12:31:08.777859 done after 0.00 mins
2020-06-15 12:31:08.779854 done after 0.

...

2020-06-15 12:31:48.987111 done after 0.00 mins
2020-06-15 12:31:48.987111 done after 0.00 mins
2020-06-15 12:31:48.987111 done after 0.00 mins
2020-06-15 12:31:49.011978 done after 0.00 mins
2020-06-15 12:31:49.013972 done after 0.00 mins
2020-06-15 12:31:49.015967 done after 0.00 mins
2020-06-15 12:31:49.018959 done after 0.00 mins
2020-06-15 12:31:49.020999 done after 0.00 mins
2020-06-15 12:31:49.023005 done after 0.00 mins
2020-06-15 12:31:49.024981 done after 0.00 mins
2020-06-15 12:31:49.026973 done after 0.00 mins
2020-06-15 12:31:49.029970 done after 0.00 mins
2020-06-15 12:31:49.032960 done after 0.00 mins
2020-06-15 12:31:49.035951 done after 0.00 mins
2020-06-15 12:31:49.039940 done after 0.00 mins
2020-06-15 12:31:49.043936 done after 0.00 mins
2020-06-15 12:31:49.050874 done after 0.00 mins
2020-06-15 12:31:49.055858 done after 0.00 mins
2020-06-15 12:31:49.061843 done after 0.00 mins
2020-06-15 12:31:49.064921 done after 0.00 mins
2020-06-15 12:31:49.064921 done after 0.

2020-06-15 12:31:49.784795 done after 0.00 mins
2020-06-15 12:31:49.784795 done after 0.00 mins
2020-06-15 12:31:49.784795 done after 0.00 mins
2020-06-15 12:31:49.784795 done after 0.00 mins
2020-06-15 12:31:49.784795 done after 0.00 mins
2020-06-15 12:31:49.784795 done after 0.00 mins
2020-06-15 12:31:49.784795 done after 0.00 mins
2020-06-15 12:31:49.784795 done after 0.00 mins
2020-06-15 12:31:49.784795 done after 0.00 mins
2020-06-15 12:31:49.800419 done after 0.00 mins
2020-06-15 12:31:49.800419 done after 0.00 mins
2020-06-15 12:31:49.800419 done after 0.00 mins
2020-06-15 12:31:49.800419 done after 0.00 mins
2020-06-15 12:31:49.821152 done after 0.00 mins
2020-06-15 12:31:49.831583 done after 0.00 mins
2020-06-15 12:31:49.837566 done after 0.00 mins
2020-06-15 12:31:49.842552 done after 0.00 mins
2020-06-15 12:31:49.849535 done after 0.00 mins
2020-06-15 12:31:49.855517 done after 0.00 mins
2020-06-15 12:31:49.861501 done after 0.00 mins
2020-06-15 12:31:49.863700 done after 0.

2020-06-15 12:31:50.656725 done after 0.00 mins
2020-06-15 12:31:50.672346 done after 0.00 mins
2020-06-15 12:31:50.677311 done after 0.00 mins
2020-06-15 12:31:50.678309 done after 0.00 mins
2020-06-15 12:31:50.682297 done after 0.00 mins
2020-06-15 12:31:50.685289 done after 0.00 mins
2020-06-15 12:31:50.690275 done after 0.00 mins
2020-06-15 12:31:50.693267 done after 0.00 mins
2020-06-15 12:31:50.698259 done after 0.00 mins
2020-06-15 12:31:50.704244 done after 0.00 mins
2020-06-15 12:31:50.709224 done after 0.00 mins
2020-06-15 12:31:50.715210 done after 0.00 mins
2020-06-15 12:31:50.721192 done after 0.00 mins
2020-06-15 12:31:50.727176 done after 0.00 mins
2020-06-15 12:31:50.736152 done after 0.00 mins
2020-06-15 12:31:50.745127 done after 0.00 mins
2020-06-15 12:31:50.748227 done after 0.00 mins
2020-06-15 12:31:50.748227 done after 0.00 mins
2020-06-15 12:31:50.763853 done after 0.00 mins
2020-06-15 12:31:50.763853 done after 0.00 mins
2020-06-15 12:31:50.779507 done after 0.

2020-06-15 12:31:51.590146 done after 0.00 mins
2020-06-15 12:31:51.590146 done after 0.00 mins
2020-06-15 12:31:51.590146 done after 0.00 mins
2020-06-15 12:31:51.611669 done after 0.00 mins
2020-06-15 12:31:51.618652 done after 0.00 mins
2020-06-15 12:31:51.628625 done after 0.00 mins
2020-06-15 12:31:51.629623 done after 0.00 mins
2020-06-15 12:31:51.631620 done after 0.00 mins
2020-06-15 12:31:51.633612 done after 0.00 mins
2020-06-15 12:31:51.635607 done after 0.00 mins
2020-06-15 12:31:51.637601 done after 0.00 mins
2020-06-15 12:31:51.639596 done after 0.00 mins
2020-06-15 12:31:51.643587 done after 0.00 mins
2020-06-15 12:31:51.647575 done after 0.00 mins
2020-06-15 12:31:51.649624 done after 0.00 mins
2020-06-15 12:31:51.652615 done after 0.00 mins
2020-06-15 12:31:51.655610 done after 0.00 mins
2020-06-15 12:31:51.658604 done after 0.00 mins
2020-06-15 12:31:51.662592 done after 0.00 mins
2020-06-15 12:31:51.663628 done after 0.00 mins
2020-06-15 12:31:51.663628 done after 0.

2020-06-15 12:31:52.506356 done after 0.00 mins
2020-06-15 12:31:52.531263 done after 0.00 mins
2020-06-15 12:31:52.534255 done after 0.00 mins
2020-06-15 12:31:52.539242 done after 0.00 mins
2020-06-15 12:31:52.542278 done after 0.00 mins
2020-06-15 12:31:52.545262 done after 0.00 mins
2020-06-15 12:31:52.549214 done after 0.00 mins
2020-06-15 12:31:52.552206 done after 0.00 mins
2020-06-15 12:31:52.555198 done after 0.00 mins
2020-06-15 12:31:52.560184 done after 0.00 mins
2020-06-15 12:31:52.566170 done after 0.00 mins
2020-06-15 12:31:52.572153 done after 0.00 mins
2020-06-15 12:31:52.580132 done after 0.00 mins
2020-06-15 12:31:52.581219 done after 0.00 mins
2020-06-15 12:31:52.581219 done after 0.00 mins
2020-06-15 12:31:52.596841 done after 0.00 mins
2020-06-15 12:31:52.596841 done after 0.00 mins
2020-06-15 12:31:52.612463 done after 0.00 mins
2020-06-15 12:31:52.628085 done after 0.00 mins
2020-06-15 12:31:52.643707 done after 0.00 mins
2020-06-15 12:31:52.659328 done after 0.

In [7]:
df

[[{'profit': 0.5,
   'loss': 0.5,
   'mean': 1.507044784121837,
   'std': 0.8804411568760298,
   'sharpe': 1.7116927944044715}],
 [{'profit': 0.5,
   'loss': 1.0,
   'mean': 1.6490038139012062,
   'std': 0.6855839105213138,
   'sharpe': 2.4052545408297488}],
 [{'profit': 0.5,
   'loss': 1.5,
   'mean': 1.446331351633323,
   'std': 0.7967640053715448,
   'sharpe': 1.8152568914792702}],
 [{'profit': 0.5,
   'loss': 2.0,
   'mean': 1.5557309572207993,
   'std': 0.6889478377199688,
   'sharpe': 2.258125901620356}],
 [{'profit': 0.5,
   'loss': 2.5,
   'mean': 1.7423590908493014,
   'std': 0.7648280508849933,
   'sharpe': 2.2781056328062146}],
 [{'profit': 0.5,
   'loss': 3.0,
   'mean': 1.7680047639086554,
   'std': 0.8396185333407398,
   'sharpe': 2.1057238420809745}],
 [{'profit': 0.5,
   'loss': 3.5,
   'mean': 1.6911868493255064,
   'std': 0.8257602324632833,
   'sharpe': 2.0480362008722715}],
 [{'profit': 0.5,
   'loss': 4.0,
   'mean': 1.6596023629944832,
   'std': 0.7443768167866079

In [8]:
len(df)

10000